In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import re
import json
WAIT = 3
SCROLL_PAUSE_SEC = 1
#options = webdriver.ChromeOptions()
#options.add_argument('headless')

In [2]:
# css 찾을때 까지 10초대기
def time_wait(num, code,driver):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait

In [56]:
def find_search_input(key_word,driver):
    # 검색창 찾기
    driver.switch_to.default_content()
    time_wait(2, 'div.input_box > input.input_search',driver)
    search = driver.find_element(By.CSS_SELECTOR,'div.input_box > input.input_search')
    search.send_keys(key_word)  # 검색어 입력
    driver.implicitly_wait(3)
    search.send_keys(Keys.ENTER)  # 엔터버튼 누르기
    sleep(5)
    #res = driver.page_source  # 페이지 소스 가져오기
    #soup = BeautifulSoup(res, 'html.parser')  # html 파싱하여  가져온다
    #sleep(1)

In [57]:
# frame 변경 메소드
def switch_frame(frame,driver):
    driver.switch_to.default_content()  # frame 초기화
    driver.implicitly_wait(1)
    driver.switch_to.frame(frame)  # frame 변경
    driver.implicitly_wait(1)



In [58]:
# 페이지 다운
def page_down(num,driver):
    body = driver.find_element(By.CSS_SELECTOR,'body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)


In [60]:
def page_up(num, driver):
    body = driver.find_element(By.CSS_SELECTOR,'body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_UP)

In [61]:
def find_review_btn(driver):
    sleep(2)
    try:
        menu_list = driver.find_elements(By.CSS_SELECTOR,'a.tpj9w')
        if len(menu_list) ==7:
            return menu_list[-1]
        else:
            return menu_list[-2]
    except:
        print('리뷰가 존재하지 않습니다.')

# 메뉴&리뷰 크롤링 함수

In [62]:
def crawling_menu_review(driver):
    # -----리뷰 버튼 클릭-------
    find_review_btn(driver).click()
    sleep(1)
    page_down(1, driver)
    driver.implicitly_wait(WAIT)
    # ----------------메뉴, 크롤링 블록----------------
    menu_list = list()
    review_list = list()
    # 메뉴, 크롤링 블록을 포함하는 try-except문
    try:
        # --------------메뉴 크롤링 블록--------------------------
        # 메뉴 요소 찾기
        menu_elements = driver.find_elements(By.CSS_SELECTOR, 'div.place_fixed_subtab > div > div:nth-child(1) > div > div > div > span > a > span:first-child')
        sleep(SCROLL_PAUSE_SEC)
        try:
            # 왼쪽 버튼 찾기
            left_btn = driver.find_element(By.CSS_SELECTOR, 'a.PznE8.aNw43')
            driver.implicitly_wait(WAIT)
            left_btn.click()
            driver.implicitly_wait(WAIT)
        except:
            pass

        try:
            # 오른족 버튼 찾기
            right_btn = driver.find_element(By.CSS_SELECTOR, 'div.place_fixed_subtab > div > div:nth-child(1) > div > div > a.ZCqf_.aNw43')
            driver.implicitly_wait(WAIT)
        except:
            pass
        # -----------여기까지--------------
        i = 0
        # menu_elentents가 존재하지 않는다면 except로 갈 것이다.
        try:
            while True:
                if menu_elements[i].text != '':  # 요소가 존재하고 메뉴 텍스트도 보일때
                    menu_list.append(menu_elements[i].text)
                    if i == len(menu_elements) - 1:  # 메뉴 리스트 길이면 나온다.
                        break
                    i += 1
                else:
                    right_btn.click()
                    sleep(SCROLL_PAUSE_SEC)
                    driver.implicitly_wait(WAIT)
        except:
            pass

        # last_scroll = driver.execute_script("return document.body.scrollHeight")
        # view_more_count = 0

        # --------------리뷰 크롤링 블록--------------------------
        try:
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                sleep(1)
                driver.implicitly_wait(SCROLL_PAUSE_SEC)

                # if view_more_count ==19:
                # break
                try:
                    # 더보기 버튼 클릭
                    driver.implicitly_wait(WAIT)
                    driver.find_element(By.CSS_SELECTOR, 'a.fvwqf').click()
                    # view_more_count += 1

                except:
                    # 더보기 버튼이 없다면 while문을 빠져나온다.
                    break
                    # 끝까지 스크롤 다운

                # new_scroll = driver.execute_script("return document.body.scrollHeight")
                # driver.implicitly_wait(WAIT)
                # if last_scroll == new_scroll:
                #    break
                # else:
                #    last_scroll= new_scroll

            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # 가게 리스트 마다 방문자 리뷰 가져오기
            review_elements = driver.find_elements(By.CSS_SELECTOR, "span.zPfVt")
            sleep(1)
            for i in review_elements:
                review_list.append(i.text)
        except:
            print('리뷰 크롤링중 오류가 났습니다.')
            pass

    except:
        print('메뉴 & 리뷰 크롤링중 오류가 났습니다.')
        pass

    return menu_list, review_list

# 전체 크롤링 함수

In [63]:
def crawling(driver, i, store_elements, review_dic):
    sleep(1)
    store_elements[i].click()  # li의 스토어를 클릭
    driver.implicitly_wait(WAIT)  # 2초 휴식
    return_name = None
    try:
        # 다른 iframe으로 변경
        switch_frame('entryIframe', driver)
        sleep(1)
        time_wait(3, 'span.Fc1rA', driver)
        try:
            # -----매장명 가져오기-----
            store_name = driver.find_element(By.CSS_SELECTOR, 'span.Fc1rA').text
            driver.implicitly_wait(2)
            review_dic[store_name] = dict()

        except:
            print('매장명 가져오기 오류')
            pass

        # -----평점 가져오기-----
        try:
            sleep(0.5)
            rank = driver.find_element(By.CSS_SELECTOR, 'div.zD5Nm.f7aZ0 > div.dAsGb > span.PXMot.LXIwF > em')
            driver.implicitly_wait(WAIT)
            review_dic[store_name]['rank'] = rank.text
        except:
            print('평점이 없습니다.')
            pass

        # --------위치 가져오기---------
        try:
            sleep(0.5)
            location = driver.find_element(By.CSS_SELECTOR, 'span.IH7VW').text
            driver.implicitly_wait(WAIT)
            review_dic[store_name]['location'] = location
        except:
            print('위치가져오기 오류')
            pass

        memu_list, review_list = crawling_menu_review(driver)

    except:
        print('크롤링 함수 오류가 났습니다.')
        pass

    # 메뉴 크롤링
    review_dic[store_name]['menu'] = memu_list

    # 방문자 리뷰 크롤링
    review_dic[store_name]['review'] = review_list
    driver.implicitly_wait(WAIT)

    return store_name

# 메인 함수

In [64]:
def main(key_word, page_num):
    review_dic = dict()
    page_num = page_num - 1
    refresh_count = 0

    url = 'https://map.naver.com/v5/search'
    driver = webdriver.Chrome('./chromedriver')
    driver.get(url)
    driver.implicitly_wait(2)

    # 검색어 입력
    find_search_input(key_word, driver=driver)
    driver.implicitly_wait(2)

    # 프레임 변경
    switch_frame('searchIframe', driver)
    driver.implicitly_wait(WAIT)

    # 페이지 버튼 찾기
    pages = driver.find_elements(By.CSS_SELECTOR, 'a.mBN2s')
    if page_num <= 5:
        pages[page_num].click()
        sleep(1)
    else:
        driver.find_elements(By.CSS_SELECTOR, 'div.zRM9F > a:nth-child(6)').click()
        driver.implicitly_wait(WAIT)
        driver.find_element(By.CSS_SELECTOR, 'div.zRM9F > a:nth-child(6)').click()
        driver.implicitly_wait(WAIT)

    driver.implicitly_wait(WAIT)
    page_down(40, driver)
    sleep(SCROLL_PAUSE_SEC)
    driver.implicitly_wait(SCROLL_PAUSE_SEC)
    page_down(5, driver)
    sleep(SCROLL_PAUSE_SEC)
    page_up(40,driver)
    sleep(SCROLL_PAUSE_SEC)
    page_up(5,driver)
    sleep(SCROLL_PAUSE_SEC)
    store_elements = driver.find_elements(By.CSS_SELECTOR, 'div.ouxiq > a:nth-child(1)')
    print('total store_list : ', len(store_elements))  # 매장리스트 접근
    # 체크포인트
    ck_pt_idx = 0

    while True:
        while True:
            try:  # 이상한 오류가 뜰경우 현재위치 까지 저장 하고, 크롤링 인덱스 반환
                store_name = crawling(driver, ck_pt_idx, store_elements, review_dic)
                print('현재 크롤링 중인 매장 인덱스와 매장명 : ', ck_pt_idx,store_name)
                ck_pt_idx += 1
                sleep(0.5)
                switch_frame('searchIframe', driver)


                # 한 refresh를 벗어나는 조건문
                if len(review_dic[store_name]['review']) >= 500:
                    break

            except:
                # json 파일로 저장
                with open(f'./Crawling2/naver_{key_word}_review_dic_page{page_num + 1}.json', 'w', encoding='utf-8') as f:
                    json.dump(review_dic, f, indent=4, ensure_ascii=False)
                return ck_pt_idx

        # 페이지의 크롤링을 나오는 조건문
        if ck_pt_idx == len(store_elements) - 1:
            break

        print('새로고침을 진행합니다.')
        driver.refresh()
        driver.implicitly_wait(1)
        switch_frame('searchIframe', driver)
        sleep(SCROLL_PAUSE_SEC)
        page_down(40, driver)
        sleep(SCROLL_PAUSE_SEC)
        driver.implicitly_wait(1)
        page_down(5, driver)
        sleep(SCROLL_PAUSE_SEC)
        store_elements = driver.find_elements(By.CSS_SELECTOR, 'div.ouxiq > a:nth-child(1)')
        print('total store_list', len(store_elements))

    print('크롤링을 저장하고, 종료합니다.')
    # json 파일로 저장
    with open(f'./Crawling2/naver_{key_word}_review_dic_page{page_num + 1}.json', 'w', encoding='utf-8') as f:
        json.dump(review_dic, f, indent=4, ensure_ascii=False)    
    

In [66]:
main('인천 횟집', 1)

C:\Users\DanielJeong\AppData\Local\Temp\ipykernel_17536\567307571.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


total store_list :  45
현재 크롤링 중인 매장 인덱스와 매장명 :  0 떡바우횟집
현재 크롤링 중인 매장 인덱스와 매장명 :  1 도쿄참치
현재 크롤링 중인 매장 인덱스와 매장명 :  2 웰컴도매횟집
현재 크롤링 중인 매장 인덱스와 매장명 :  3 떡바우횟집
현재 크롤링 중인 매장 인덱스와 매장명 :  4 동남횟집
현재 크롤링 중인 매장 인덱스와 매장명 :  5 경북횟집
현재 크롤링 중인 매장 인덱스와 매장명 :  6 김대통의대통물회&횟집
현재 크롤링 중인 매장 인덱스와 매장명 :  7 삼천포횟집
크롤링 함수 오류가 났습니다.


8

In [11]:
with open('./naver/naver_울산 횟집_review_dic_page1.json', encoding='utf-8') as f:
    ex = json.load(f)
len(ex)

39

In [24]:
ex.keys()

dict_keys(['떡바우횟집', '웰컴도매횟집', '일차로물회', '동남횟집', '경북횟집', '김대통의대통물회&횟집', '삼천포횟집', '육일횟집', '뽀빠이물회', '대복횟집', '강동회수산', '독도참가자미', '삼태육경 회 타운', '만원의행복횟집', '송가정횟집', '뱃고동횟집', '금바다횟집', '진주횟집', '금광횟집', '신명횟집', '야무야무', '방어진정자횟집', '해강횟집', '대게신도시횟집', '장원횟집', '진하자연산횟집', '남일횟집', '갯바위횟집', '바닷가횟집', '미미정횟집고래고기', '청파', '일월횟집', '대어횟집', '후포미송', '섬뜰', '부산횟집', '수암사랑도횟집', '한양횟집', '동남물회'])

In [56]:
ex.keys()

dict_keys(['떡바우횟집', '웰컴도매횟집', '일차로물회', '동남횟집', '경북횟집', '김대통의대통물회&횟집', '삼천포횟집', '육일횟집', '뽀빠이물회', '대복횟집', '강동회수산', '독도참가자미', '삼태육경 회 타운', '만원의행복횟집', '송가정횟집', '뱃고동횟집', '금바다횟집', '진주횟집', '금광횟집', '신명횟집', '야무야무', '방어진정자횟집', '해강횟집', '대게신도시횟집', '장원횟집', '진하자연산횟집', '남일횟집', '갯바위횟집', '바닷가횟집', '미미정횟집고래고기', '청파', '일월횟집', '대어횟집', '후포미송', '섬뜰', '부산횟집', '수암사랑도횟집', '한양횟집', '동남물회'])

In [28]:
ex['삼천포횟집']

{'rank': '4.19',
 'location': '울산 동구 주전해안길 278',
 'menu': ['물회',
  '전복',
  '미역국',
  '전복죽',
  '육회',
  '매운탕',
  '커피',
  '국수',
  '멍게',
  '덮밥',
  '매실차',
  '소라'],
 'review': ['찬은 정갈하니 맛 괜찮았고 일반물회는 약간 매콤하게 맛있게 잘먹었어요 종업원분이 굉장히 친절하셔서 기분 좋게 이용했답니다^^',
  '푸짐하고 신선합니다 주차 편하고 친절하세요 음식 맛 너무 좋고 국수 밥 추가 할 수 있어 좋아요 추천합 니다~~!! (해산물에 해삼이랑 전복 뿐인듯요 ...',
  '자주가던 단골집 이제 이곳으로 옮기려구요. 바쁘고 장사 잘되는집은 항상 불친절한 경우가 많던데 이곳 은 친절하고 주차하기 좋고 양도많고, 반찬도 ...',
  '직원분들이 친절하시고 현지분들이 많이 오는거 같\n았어요. 저는 개인적으로 고추장텁텁한맛을 싫어해\n서였는지 그냥 그랬고, 같이간분들은 맛있게 ...',
  '친절하시고 육회가 맛있어요! 달콤새콤 짱',
  '커피가 맛나고\n주변 뷰가 좋아요',
  '대기시간은조금있었지만\n맛있었어요~~ 친절하기도하고\n또 먹고싶네요:)',
  '맛있는데 대기줄이기네요',
  '예전에 비해 육수가 조금 다름..고추장맛이 너무세진것같음',
  '제철메뉴들이 있어서 아쉬웠어요ㅠㅜ\n매운탕도요 ㅠ 맛은 있어요 굿',
  '물회 맛집이라더니 재료도 싱싱하고 맛있어요!다만 조금 매워서;;; 그거말곤 진짜 맛있어요',
  '매우 맛있어요',
  '맛있는데 대기가 길어요',
  '물회는 역시 육일입니다',
  '좋아요',
  '가성비 최고입니다',
  '덮밥보다 물회에 한 표. 회 자체가 맛있다기보다 그냥 무난하게 한그릇 잘 먹었어요. 오히려 미역국과 전이 맛났어요.',
  '물회는 육일!',
  '물회 최애맛집!!',
  '감사합니다',
  '굿',
  '특별히 맛있다기보단 다 아는 물회맛이지만 가게가 깨끗하고 반찬들도 정갈해서 주전

In [38]:
review_dic = dict()
key_word = '울산 횟집'
page_num  = 0
refresh_count = 0

url = 'https://map.naver.com/v5/search'
driver = webdriver.Chrome('./chromedriver')
driver.get(url)
driver.implicitly_wait(2)

# 검색어 입력
find_search_input(key_word,driver=driver)
driver.implicitly_wait(2)

#프레임 변경
switch_frame('searchIframe',driver)
driver.implicitly_wait(WAIT)

# 페이지 버튼 찾기
pages = driver.find_elements(By.CSS_SELECTOR,'a.mBN2s')
if page_num <= 5:
    pages[page_num].click()
    driver.implicitly_wait(WAIT)
else:
    driver.find_elements(By.CSS_SELECTOR,'div.zRM9F > a:nth-child(6)').click()
    driver.implicitly_wait(WAIT)
    driver.find_element(By.CSS_SELECTOR, 'div.zRM9F > a:nth-child(6)').click()
    driver.implicitly_wait(WAIT)

driver.implicitly_wait(WAIT)
page_down(40,driver)
sleep(1)
driver.implicitly_wait(1)
page_down(5,driver)
sleep(2)
driver.execute_script("window.scrollTo(0, 0);")
driver.implicitly_wait(WAIT)
store_elements = driver.find_elements(By.CSS_SELECTOR,'div.ouxiq.icT4K > a:nth-child(1)')
print('total store_list : ', len(store_elements)) #매장리스트 접근
# 체크포인트

C:\Users\JSWonner\AppData\Local\Temp\ipykernel_8592\2411810518.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


total store_list :  44


In [39]:
switch_frame('searchIframe',driver)

In [40]:
store_name = crawling(driver, 43, store_elements, review_dic)

In [42]:
store_name

'동화회시장'

In [44]:
driver.refresh()

In [45]:
switch_frame('searchIframe',driver)

In [47]:
page_down(40,driver)

In [26]:
store_elements = driver.find_elements(By.CSS_SELECTOR,'div.ouxiq.icT4K > a:nth-child(1)')

In [28]:
store_name = crawling(driver, 40, store_elements, review_dic)

In [29]:
store_name

'수암사랑도횟집'

In [ ]:
crawling(driver, 4, store_elements, review_dic)

In [30]:
review_dic

{'떡바우횟집': {'rank': '4.41',
  'location': '울산 울주군 서생면 신암해안길 13 2층',
  'menu': ['물회', '매운탕', '밥집', '회덮밥', '농어', '광어'],
  'review': ['음식이 간결하고 맛있네요 ^^',
   '울산여행왔다가 근처 밥집찾아서 갔는데 넘 맛있게\n잘먹었어요~ 물회👍🏻👍🏻',
   '오랫만에 방문했는데 맛있게 잘 먹었습니다~',
   '물회 최고',
   '매운탕은 비리네요\n회덮밥은 먹을만해요',
   '친절하고 좋아요 ^^',
   '굿',
   '맛있어요',
   '굿',
   '맛있네요',
   '맛있네요',
   '신선합니다',
   '좋아요~',
   '좋음',
   '회가 신선합니다.',
   '가끔 가는곳',
   '맛있어요',
   '회 ㅡㅡ굿입니다 일하시는분들 모두친절하시구요',
   '굿',
   '가성비좋음',
   '횟밥 가성비 최고 추천',
   '알고 있는 맛',
   '굿',
   '농어랑 광어 포장해와서 맛있게 잘 먹었네요.\n농어도 광어도 맛 좋드라구요!!',
   '굿',
   '맛있어요',
   '맛있어요!',
   '😀😀😀😀😀',
   '좋아요',
   '맛있어요\n횟밥시키면 매운탕 도 같이 줘요!!!',
   '',
   'ㆍ',
   '회 굳']},
 '동화회시장': {'rank': '4.44',
  'location': '울산 남구 중앙로44번길 11',
  'menu': [],
  'review': ['오랜만에 방문 \n맛있게 잘 먹었어요',
   '주차장이없어서 아쉽지만 맛잇게 잘먹음',
   '좋아요',
   '동네 횟집\n룸이 있어 모임하기 좋아요.',
   '가족모임있었는데 음식 맛있고 아이 있으니 계란찜등 아이가 먹을 만한 음식도 빨리 챙겨주셔서 좋았어요',
   'ㅇ',
   '좋아요',
   '첫방문인데 아주 만족스럽네요. 재방문의사 있습니다. 번창하십시요',
   '최고체고',
   '좋아요',
   '맛나게 먹고 왔어요',


In [32]:
del review_dic